In [ ]:
import pandas as pd 
import time 
import seaborn as sns
import numpy as  np
from dateutil.parser import parse as parse_date
from matplotlib import pyplot as plt
import datetime

%matplotlib inline

In [ ]:
#install spotipy as its not on colab
!pip install spotipy 

In [ ]:
#add details to connect to API
USERNAME = '' #your spotify username
CLIENT_ID = "" #set at your developer account
CLIENT_SECRET = "" #set at your developer account
REDIRECT_URI =  'http://localhost:8000/' #set at your developer account, usually "http://localhost:8000"
SCOPE = "user-read-private user-read-email user-read-playback-state user-read-currently-playing user-library-read" # or else

In [ ]:
## log in my credentials for API
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data
import spotipy.util as util

sp = spotipy.Spotify() 
from spotipy.oauth2 import SpotifyClientCredentials 
cid ="" 
secret = "" 
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 
sp.trace=False 

In [ ]:
#select a 'playlist' (all my liked songs)
playlist = sp.user_playlist("", "")

In [ ]:
playlist

In [ ]:
#iterate through the playlistand append every item. 
good_tracks_1 = playlist["tracks"]
good_songs_1 = good_tracks_1["items"] 
while good_tracks_1['next']:
    good_tracks_1 = sp.next(good_tracks_1)
    for item in good_tracks_1["items"]:
        good_songs_1.append(item)
good_ids = [] 
print(len(good_songs_1))
for i in range(len(good_songs_1)):
    good_ids.append(good_songs_1[i]['track']['id'])
print(len(good_tracks_1))

data = pd.DataFrame()

data['added_at'] = list()
added_at = []
ident = []

for i in good_songs_1:
    added_at.append(i['added_at'])
    ident.append(i['track']['id'])
    
data['added_at'] = added_at
    
    

In [ ]:
data['added_at'].value_counts()

In [ ]:
#remove weird null from list
good_ids = [i for i in good_ids if i]

In [ ]:
#Here is where we get all of the audio_features for the tracks on the good playlists.
features = []
inSavedTracks = []
j = 0
for i in range(0,len(good_ids),50):
    audio_features = sp.audio_features(good_ids[i:i+50])
    for track in audio_features:
        features.append(track)
        track = good_songs_1[j]
        j= j+1
        features[-1]['Artist'] =  track['track']['artists'][0]['name']
        features[-1]['TrackName'] =  track['track']['name']
        features[-1]['ReleaseDate'] =   pd.to_datetime(track['track']['album']['release_date']) if track['track']['album']['release_date'] else None     
        features[-1]['trackPopularity'] = track['track']['popularity']
        features[-1]['target'] = 1

In [ ]:
##turn into a dataframe for analysis
featurelist = pd.DataFrame(features)

In [ ]:
featurelist.head()

In [ ]:
## now remove any uncessary columns
featurelist.drop(columns=['id', 'key', 'mode','type', 'uri','track_href','analysis_url', 'duration_ms', 'time_signature'])

In [ ]:
##show averages across featurelist
featurelist.describe()

In [ ]:
##numerical features 
num_features = featurelist[['danceability','energy','speechiness','acousticness','instrumentalness', 'liveness', 'valence']]

In [ ]:
##plot the mean of every feature across all songs, sho which songs i have most 

fig = num_features.mean().plot(kind='barh')

In [ ]:
##show the most danceable song in my list
ordered = featurelist[['Artist', 'TrackName', 'danceability','energy','speechiness','acousticness','instrumentalness', 'liveness', 'valence']]
grouped2 = ordered.sort_values('danceability', ascending=False)
grouped2.head(20)

In [ ]:
#show least energetic songs in my list.
ordered = featurelist[['Artist', 'TrackName', 'danceability','energy','speechiness','acousticness','instrumentalness', 'liveness', 'valence']]
grouped2 = ordered.sort_values('energy', ascending=True)
grouped3 = grouped2.iloc[:]
grouped3.head(10)

In [ ]:
print(len(featurelist))

In [ ]:
danceable = ordered.sort_values('danceability', ascending=False)

In [ ]:
danceable.head(15)

In [ ]:
danceable.to_csv('/Users/jackohagan/Documents/datascience/Spotify/songnamedata.csv')